In [1]:
%%writefile sa.c
/* Blue Waters Petascale Semester Curriculum v1.0
 * Unit 4: OpenMP
 * Lesson 10: Ensemble Based Simulated Annealing in OpenMP
 * sa.c
 * Developed by David A. Joiner for the Shodor Education Foundation, Inc.
 *
 * Copyright (c) 2020 The Shodor Education Foundation, Inc.
 *
 * Browse and search the full curriculum at
 * <http://shodor.org/petascale/materials/semester-curriculum>.
 *
 * We welcome your improvements! You can submit your proposed changes to this
 * material and the rest of the curriculum in our GitHub repository at
 * <https://github.com/shodor-education/petascale-semester-curriculum>.
 *
 * We want to hear from you! Please let us know your experiences using this
 * material by sending email to petascale@shodor.org
 *
 * This program is free software: you can redistribute it and/or modify
 * it under the terms of the GNU Affero General Public License as published
 * by the Free Software Foundation, either version 3 of the License, or
 * (at your option) any later version.
 *
 * This program is distributed in the hope that it will be useful,
 * but WITHOUT ANY WARRANTY; without even the implied warranty of
 * MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
 * GNU Affero General Public License for more details.
 *
 * You should have received a copy of the GNU Affero General Public License
 * along with this program.  If not, see <https://www.gnu.org/licenses/>.
 */

#include <stdio.h>
#include <stdlib.h>
#include <math.h>
#include <time.h>
#include <unistd.h>
#include <omp.h>


double fake_delay(int n) {
    int i,j;
    double x = 1.0;

    for(i=0;i<n;i++) {
        for(j=0;j<n;j++) {
           x *= (i+j);
        }
    }
    return 0*x+1;

}

double funcN(int n, double *x_arr, void * v_opts) {
    double r=0;
    int i;
    double retval;
    for(i=0;i<n;i++) {
        r += x_arr[i]*x_arr[i];
    }
    r = sqrt(r);
    retval= fake_delay(100);
    for(i=0;i<n;i++) {
        retval *= sin(x_arr[i]);
    }
    retval *= exp(-r*r);
    retval += 1.0e-20*r*r;
    return retval;
}

double func(int n, double * x_arr, void * v_opts) {
     // sample function
     double x = x_arr[0];
     double y;
     y = x_arr[1];
     double f = -sin(x)*sin(y)*exp(-(x*x+y*y))+1.0e-20*(x*x+y*y);
     //double f = x*x+y*y;
     return f;
}



void print_array(int n, double * x) {
    int i;
    for(i=0;i<n;i++) {
        printf("%lf ",x[i]);
    }
    printf("\n");
}

double drand_range(double min, double max) {
    double rand_value =  min+(double)rand()/RAND_MAX*(max-min);
    return rand_value;
}

typedef struct {
    double energy;
    double temp;
    double step;
    double target;
    double allowance;
    double step_factor;
    double cooling_factor;
    double epsilon;
    int n;
    int max_func_calls;
    int num_func_calls;
    int ntrial;
    double * x;
    double * trial_x;
    double * dx;
    int energy_initialized;
} SAstruct;

void SA_init(SAstruct * model, int n) {
    model->temp=1.0;
    model->step=0.1;
    model->target=0.5;
    model->allowance=0.1;
    model->step_factor=0.95;
    model->cooling_factor=0.95;
    model->epsilon=1.0e-7;
    model->max_func_calls = 10000000;
    model->num_func_calls = 0;
    model->n = n;
    model->ntrial = 100;
    model->x = (double *)calloc(sizeof(double),n);
    model->trial_x = (double *)calloc(sizeof(double),n);
    model->dx = (double *)calloc(sizeof(double),n);
    model->energy_initialized=0;
}

void SA_free(SAstruct * model) {
    free(model->x);
    free(model->trial_x);
    free(model->dx);
}


int SA_step(SAstruct * model, double func(int, double *, void *), void * opts) {
    int i;
    double trial_energy;

    if(!model->energy_initialized) {
        model->energy = func(model->n,model->x,opts);
        model->energy_initialized = 1;
    }

    // pick a random step;
    for(i=0;i<model->n;i++) {
        model->dx[i] = drand_range(-model->step,model->step);
    }

    // try out new value
    for(i=0;i<model->n;i++) {
        model->trial_x[i] = model->x[i] + model->dx[i];
    }
    trial_energy = func(model->n,model->trial_x,opts);
    model->num_func_calls++;
    

    double deltaE = trial_energy-model->energy;
    if(deltaE<0) {
        for(i=0;i<model->n;i++) {
            model->x[i] = model->trial_x[i];
            model->energy = trial_energy;
        }
        return 1;
    } else {
        if(drand_range(0.0,1.0)<exp(-deltaE/model->temp)) {
            for(i=0;i<model->n;i++) {
                model->x[i] = model->trial_x[i];
                model->energy = trial_energy;
            }
            return 1;
        } else {
            return 0;
        }
    }
    
}

int SA_adjust_step(SAstruct * model,
        double func(int, double *, void *), void * opts) {
    int iter,i; 
    int count_success = 0;
    double ratio;

    for(iter=0;iter<model->ntrial;iter++) {
        count_success += SA_step(model,func,opts);
    }
    
    ratio = (double)count_success/model->ntrial;
    if(ratio<model->target-model->allowance) {
        // too greedy, make step smaller
        model->step *= model->step_factor;
        return -1;
    } else if(ratio>model->target+model->allowance) {
        // too timid, make step larger
        model->step /= model->step_factor;
        return 1;
    } else {
        // just right
        return 0;
    }
    
}


int SA_check_temperature(SAstruct * model,
        double func(int, double *, void *), void * opts) {

    int probe;

    probe = SA_adjust_step(model,func,opts);
    if(probe==0) {
        // thermal equilibrium reached, drop temperature
        model->temp*= model->cooling_factor;
        if(model->step<model->epsilon) {
            // if step size small at thermal equil, converge
            return 1;
        }
    }
    // check for convergence failure
    if(model->num_func_calls>model->max_func_calls) {
        printf("WARNING, FAILURE TO CONVERGE IN SA\n");
        return 2;
    }
    return 0;
    
}

int main(int argc, char ** argv) {
    int i;
    double start,end;
    SAstruct model;
    SA_init(&model,10);

    int seed = time(NULL);
    printf("SEED %d\n", seed);
    srand(seed);

    // initialize guess
    for(i=0;i<model.n;i++) {
        model.x[i] = 1.0;
    }

    start = omp_get_wtime();
    while(!SA_check_temperature(&model,funcN,NULL)) {
        // while not converged
            // while not at thermal equilibrium
                // adjust step
            // reduce temperature
    }
    end = omp_get_wtime();

    // print output
    printf("SOLUTION %lf %lf\n",model.x[0],model.x[1]);
    printf("TOTAL CALLS %d\n",model.num_func_calls);
    printf("Time to complete %lf\n",(end-start));

    SA_free(&model);
}


Overwriting sa.c


In [2]:
!gcc -o sa sa.c -lm -fopenmp


In [3]:
!./sa

SEED 1594484021
SOLUTION 0.653271 -0.653271
TOTAL CALLS 208400
Time to complete 7.267515



#Simulated Annealing

The simulated annealing algorithm is known as a method for finding the optimum of a multi-valued function $f(\vec{x})$. The process proceeds by having an "annealer" which is able to take a random walk in $\vec{x}$, guided such that steps that reduce the value of $f(\vec{x})$ are preferred. The degree to which steps that increase $f(\vec{x})$ are allowed are controlled by a "temperature" parameter $T$ which is gradually reduced over the course of the minimization.

The classic algorithm involves a real valued function of real valued dependent variables. Steps are taken with some step size in that space, which might be done one dimension at a time or might be done in all dimensions at once. Each step is subjected to a Metropolis Hasting's algorithm, with an acceptance probability of

$$
a(\vec{x}_{new},\vec{x}_{old}) = max(1,exp[(f(\vec{x}_{new})-f(\vec{x}_{old}))/T)
$$.

The step size is adjusted automatically in batches of size $n_{batch}$, so that the frcation of accepted steps is within some preset threshhold of a desired ratio (typically $50\% \pm 10\%$).
When this has been achieved, the temperature is then allowed to "cool" by a small amount, and the random walk proceeds iteratively until the temperature and step size are both small.

#Serial Code Overview

The sa.c code provided has a serial implementation of a simulated annealing optimization. Sample functions are provided for optimization, a simple 2D function with multiple minimums, and an extension of the same test function to N dimensions. An additional delay function is added to simulate the effect of optimizing a problem which requires substantially more cpu time per function call.

The main routine contains a primary iteration loop on SA_check_temperature.

<PRE>
    while(!SA_check_temperature(&model,funcN,NULL)) {
        // while not converged
            // while not at thermal equilibrium
                // adjust step
            // reduce temperature
    }
</PRE>

SA_check_temperature, in return, runs a batch of random walks to determine temperature convergence. It does this using the method SA_adjust_step.

<PRE>
    probe = SA_adjust_step(model,func,opts);
    if(probe==0) {
        // thermal equilibrium reached, drop temperature
        model->temp*= model->cooling_factor;
        if(model->step<model->epsilon) {
            // if step size small at thermal equil, converge
            return 1;
        }
    }
</PRE>

SA_adjust_step, in turn, runs a batch of steps and determines the number of times a step has been successful.

<PRE>
    for(iter=0;iter<model->ntrial;iter++) {
        count_success += SA_step(model,func,opts);
    }

    ... (additional logic related to use of count_success)
</PRE>

The number of successful steps is used to determine whether or how to adjust the step size.

A single step is calculated in the SA_step method, with state information on the process passed in the model structure.

<PRE>
int SA_step(SAstruct * model, double func(int, double *, void *), void * opts) {
    int i;
    double trial_energy;

    if(!model->energy_initialized) {
        model->energy = func(model->n,model->x,opts);
        model->energy_initialized = 1;
    }

    // pick a random step;
    for(i=0;i<model->n;i++) {
        model->dx[i] = drand_range(-model->step,model->step);
    }

    // try out new value
    for(i=0;i<model->n;i++) {
        model->trial_x[i] = model->x[i] + model->dx[i];
    }
    trial_energy = func(model->n,model->trial_x,opts);
    model->num_func_calls++;
    

    double deltaE = trial_energy-model->energy;
    if(deltaE<0) {
        for(i=0;i<model->n;i++) {
            model->x[i] = model->trial_x[i];
            model->energy = trial_energy;
        }
        return 1;
    } else {
        if(drand_range(0.0,1.0)<exp(-deltaE/model->temp)) {
            for(i=0;i<model->n;i++) {
                model->x[i] = model->trial_x[i];
                model->energy = trial_energy;
            }
            return 1;
        } else {
            return 0;
        }
    }   
}
</PRE>

Note that nowhere in this process is there any clear concurrency--this is fundamentally tracking of a single random walk process, with loop carried dependency from one step to another.

#Student Instructions.

1. Compile and run sa.c. Vary the value of N, and investigate the total running time and number of iterations required for convergence as a function of the dimensionality of the problem. How does the number of iterations required to converge change with the number of free parameters?

2. Looking at the two key loops in the code, the first being the while loop in the main routine and the second being the for loop in SA_adjust step, do you see any concurrency that can be described? Discuss the loop carried dependencies present in both loops and throughout the SA algorithm.

## Ensemble Based Simulated Annealing

A method that is used for parallelization of the SA algorithm is to modify the algorithm to introduce concurrency in the determination of step size. Instead of a single random walker moving ntrials steps, multiple walkers use fewer steps, with the total number of steps maintained between the walkers. Each random walk process is shorter, and at some point a choice must be made as to which of the walkers will continue to the next iteration, so the total number of steps per convergence used is less--this potentially represents a different algorithm.

This modified algorithm is Ensemble-Based Simulated Annealing (EBSA). It parallelizes a single random walk that is $n_{batch}$ long and replaces it with $n_{thread}$ random walks that are $n_{batch}/n_{thread}$ long. 

3. List at least 3 concerns regarding how this algorithmic change might affect convergence, and regarding how one would need to implement a change of multiple annealers instead of a single annealer in the SA_adjust_step method.

4. Compare the ebsa.c code to the sa.c code. Note the change of random number generator from rand() to drand48_r(). Look up the definition of drand48_r, and describe why this change is necessary. Why is it necessary to keep an array of state variables for random number generators?

5. Compare the ebsa.c code to the sa.c code. Note the use of a global memory structure and copy routine. Describe why you can not simply declare the memory structure to be private.

6. Compile and run ebsa.c, and compare ebsa.c to sa.c for equivalent problems. Do this for varying values of N. Consider also changing the artificial delay in funcN. How does the parallel solution of ebsa.c compare to sa.c? Support your answer with simulation results.




#Teacher instructions:

Students should not that there is a loop carried dependency throughout the problem, and that there is no way to parallelize the algorithm without changing it.

In changing the algorithm, discuss with the students the need for thorough testing--if the algorithm is different the solution is potentially different. Especially given that this will effectively "shorten" the most successful walk, which potentially could be equivalent to training too fast.

Note that this is a random algorithm, and that multiple tests must be done to ensure consistency.

Discuss with students in which cases it might be more prudent to use parallelism to perform a large ensemble of complete runs, as opposed to speeding up a single run, given the random nature of the results.

##Thread safe random number generation.

When using random numbers in shared memory threaded programs, be sure to use a thread safe implementation of your random number generator. 

In C, you will want to use drand48_r (reentrant) instead of rand, and the accompanying srand48_r instead of srand. 

https://man7.org/linux/man-pages/man3/drand48_r.3.html

Depending on your code structure, this may require storage of multiple random number streams for each thread. If this is occuring outside of a single #pragma omp parallel block, that would require the use of some passed or global memory structure.

Note the use of global <PRE>struct drand48_data randThread[MAX_THREADS];</PRE> in the ebsa.c code, initialized in main() and used, along with a call to omp_thread_num() in each random number call.

##Custom memory structures

For anything more than simple typed variable, you will need to provide a method of initializing shared data structures across threads.

When the annealing problem splits into threads to determine step size, separate copies of the annealer's data structure are needed for each thread, initialized from the current state. Note that a copy routine is written and provided for this purpose, and that memory is allocated and stored as a global variable for multiple copies of the annealer data structure.

<PRE>SAstruct localModel[MAX_THREADS];</PRE>


In [7]:
%%writefile ebsa.c
/* Blue Waters Petascale Semester Curriculum v1.0
 * Unit 4: OpenMP
 * Lesson 10: Ensemble Based Simulated Annealing in OpenMP
 * ebsa.c
 * Developed by David A. Joiner for the Shodor Education Foundation, Inc.
 *
 * Copyright (c) 2020 The Shodor Education Foundation, Inc.
 *
 * Browse and search the full curriculum at
 * <http://shodor.org/petascale/materials/semester-curriculum>.
 *
 * We welcome your improvements! You can submit your proposed changes to this
 * material and the rest of the curriculum in our GitHub repository at
 * <https://github.com/shodor-education/petascale-semester-curriculum>.
 *
 * We want to hear from you! Please let us know your experiences using this
 * material by sending email to petascale@shodor.org
 *
 * This program is free software: you can redistribute it and/or modify
 * it under the terms of the GNU Affero General Public License as published
 * by the Free Software Foundation, either version 3 of the License, or
 * (at your option) any later version.
 *
 * This program is distributed in the hope that it will be useful,
 * but WITHOUT ANY WARRANTY; without even the implied warranty of
 * MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
 * GNU Affero General Public License for more details.
 *
 * You should have received a copy of the GNU Affero General Public License
 * along with this program.  If not, see <https://www.gnu.org/licenses/>.
 */

#include <stdio.h>
#include <stdlib.h>
#include <math.h>
#include <time.h>
#include <unistd.h>
#include <omp.h>

#define MAX_THREADS 8



double fake_delay(int n) {
    int i,j;
    double x = 1.0;

    for(i=0;i<n;i++) {
        for(j=0;j<n;j++) {
           x *= (i+j);
        }
    }
    return 0*x+1;

}

double funcN(int n, double *x_arr, void * v_opts) {
    double r=0;
    int i;
    double retval;
    for(i=0;i<n;i++) {
        r += x_arr[i]*x_arr[i];
    }
    r = sqrt(r);
    retval= fake_delay(100);
    for(i=0;i<n;i++) {
        retval *= sin(x_arr[i]);
    }
    retval *= exp(-r*r);
    retval += 1.0e-20*r*r;
    return retval;
}

double func(int n, double * x_arr, void * v_opts) {
     // sample function
     double x = x_arr[0];
     double y;
     y = x_arr[1];
     double f = -sin(x)*sin(y)*exp(-(x*x+y*y))+1.0e-20*(x*x+y*y);
     //double f = x*x+y*y;
     return f;
}

void print_array(int n, double * x) {
    int i;
    for(i=0;i<n;i++) {
        printf("%lf ",x[i]);
    }
    printf("\n");
}


typedef struct {
    double energy;
    double temp;
    double step;
    double target;
    double allowance;
    double step_factor;
    double cooling_factor;
    double epsilon;
    int n;
    int max_func_calls;
    int num_func_calls;
    int ntrial;
    double * x;
    double * trial_x;
    double * dx;
    int energy_initialized;
} SAstruct;

SAstruct localModel[MAX_THREADS];
struct drand48_data randThread[MAX_THREADS];

double drand_range(double min, double max) {
    double randCall;
    drand48_r(&(randThread[omp_get_thread_num()]),&randCall);
    double rand_value =  min+(max-min)*randCall;
    return rand_value;
}
void SA_copy(SAstruct * source, SAstruct * dest) {
    dest->energy = source->energy;
    dest->temp = source->temp;
    dest->step = source->step;
    dest->target = source->target;
    dest->allowance = source->allowance;
    dest->step_factor = source->step_factor;
    dest->cooling_factor = source->cooling_factor;
    dest->epsilon = source->epsilon;
    dest->n = source->n;
    dest->max_func_calls = source->max_func_calls;
    dest->num_func_calls = source->num_func_calls;
    dest->ntrial = source->ntrial;
    dest->energy_initialized = source->energy_initialized;
    for(int i=0;i<source->n;i++) {
        dest->x[i]=source->x[i];
        dest->trial_x[i]=source->trial_x[i];
        dest->dx[i]=source->dx[i];
    }
}

void SA_init(SAstruct * model, int n) {
    model->temp=1.0;
    model->step=0.1;
    model->target=0.5;
    model->allowance=0.1;
    model->step_factor=0.95;
    model->cooling_factor=0.95;
    model->epsilon=1.0e-7;
    model->max_func_calls = 10000000;
    model->num_func_calls = 0;
    model->n = n;
    model->ntrial = 100;
    model->x = (double *)calloc(sizeof(double),n);
    model->trial_x = (double *)calloc(sizeof(double),n);
    model->dx = (double *)calloc(sizeof(double),n);
    model->energy_initialized=0;
}

void SA_free(SAstruct * model) {
    free(model->x);
    free(model->trial_x);
    free(model->dx);
}


int SA_step(SAstruct * model, double func(int, double *, void *), void * opts) {
    int i;
    double trial_energy;

    if(!model->energy_initialized) {
        model->energy = func(model->n,model->x,opts);
        model->energy_initialized = 1;
    }

    // pick a random step;
    for(i=0;i<model->n;i++) {
        model->dx[i] = drand_range(-model->step,model->step);
    }

    // try out new value
    for(i=0;i<model->n;i++) {
        model->trial_x[i] = model->x[i] + model->dx[i];
    }
    trial_energy = func(model->n,model->trial_x,opts);
    model->num_func_calls++;
    

    double deltaE = trial_energy-model->energy;
    if(deltaE<0) {
        for(i=0;i<model->n;i++) {
            model->x[i] = model->trial_x[i];
            model->energy = trial_energy;
        }
        return 1;
    } else {
        if(drand_range(0.0,1.0)<exp(-deltaE/model->temp)) {
            for(i=0;i<model->n;i++) {
                model->x[i] = model->trial_x[i];
                model->energy = trial_energy;
            }
            return 1;
        } else {
            return 0;
        }
    }
    
}

int SA_adjust_step(SAstruct * model,
        double func(int, double *, void *), void * opts) {
    int iter,i; 
    int count_success = 0;
    double ratio;
    int num_func_calls = model->num_func_calls;
    int add_func_calls = 0;
    SAstruct * threadModel;
    double bestEnergy = model->energy;

    // NEED TO: split our model into n_threads different models
#pragma omp parallel private(threadModel) reduction(+:count_success) \
    reduction(+:add_func_calls)
{
    threadModel = &(localModel[omp_get_thread_num()]);
    SA_copy(model,threadModel);
// each walker does part of the random walk
#pragma omp for schedule(static)
    for(iter=0;iter<model->ntrial;iter++) {
        count_success += SA_step(threadModel,func,opts);
    }
    add_func_calls += threadModel->num_func_calls - num_func_calls;
// keep best walker
    if(threadModel->energy<bestEnergy) {
#pragma omp critical
      {
        bestEnergy = threadModel->energy;
        SA_copy(threadModel,model);
      }
    }
}
    model->num_func_calls = num_func_calls + add_func_calls;
    
    ratio = (double)count_success/model->ntrial;
    if(ratio<model->target-model->allowance) {
        // too greedy, make step smaller
        model->step *= model->step_factor;
        return -1;
    } else if(ratio>model->target+model->allowance) {
        // too timid, make step larger
        model->step /= model->step_factor;
        return 1;
    } else {
        // just right
        return 0;
    }
    
}


int SA_check_temperature(SAstruct * model,
        double func(int, double *, void *), void * opts) {

    int probe;

    probe = SA_adjust_step(model,func,opts);
    if(probe==0) {
        // thermal equilibrium reached, drop temperature
        model->temp*= model->cooling_factor;
        if(model->step<model->epsilon) {
            // if step size small at thermal equil, converge
            return 1;
        }
    }
    // check for convergence failure
    if(model->num_func_calls>model->max_func_calls) {
        printf("WARNING, FAILURE TO CONVERGE IN SA\n");
        return 2;
    }
    return 0;
    
}

int main(int argc, char ** argv) {
    int i;
    double start,end;
    SAstruct model;
    SA_init(&model,10);

    int seed = time(NULL);
    printf("SEED %d\n", seed);
#pragma omp parallel
    srand48_r(seed+omp_get_thread_num(),&(randThread[omp_get_thread_num()]));

    // initialize guess
    for(i=0;i<model.n;i++) {
        model.x[i] = 1.0;
    }

    for(int i=0;i<MAX_THREADS;i++) { 
        SA_init(&(localModel[i]),model.n);
    }

    start = omp_get_wtime();
    while(!SA_check_temperature(&model,funcN,NULL)) {
        // while not converged
            // while not at thermal equilibrium
                // adjust step
            // reduce temperature
    }
    end = omp_get_wtime();

    // print output
    printf("SOLUTION %lf %lf\n",model.x[0],model.x[1]);
    printf("TOTAL CALLS %d\n",model.num_func_calls);
    printf("Time to complete %lf\n",(end-start));

    SA_free(&model);
    for(int i=0;i<MAX_THREADS;i++) { 
        SA_free(&(localModel[i]));
    }
}


Overwriting ebsa.c


In [8]:
!gcc -o ebsa ebsa.c -lm -fopenmp

In [9]:
!./ebsa

SEED 1594484148
SOLUTION 0.653271 0.653271
TOTAL CALLS 201500
Time to complete 4.090362
